# SQLAlchemy CRUD Lab

In this lab we will once again use SQLAlchemy to create a database and add a table. We will also work with a session to insert data into the table, update data already on the table, and delete data from the table. By the end of the lab, we will be comfortable using sessions to interact with and make changes to the table in our database.

### Objective

* Become comfortable performing CRUD (Create, Read, Update, Delete) actions with SQLAlchemy

### Sessions

Recall that the `Session` establishes all communication with the database.  The `Session` object follows the [Unit of Work](https://martinfowler.com/eaaCatalog/unitOfWork.html) pattern for interacting with the database.  It keeps track of all changes separately in an ongoing transaction until we explicitly commit the changes to the database.  In other words, the session logs all the changes we make separately, whether they be table inserts, updates, or deletes, until we tell it to **flush** the session with `session.commit()`.

As we saw in the previous lab, we create a session by establishing a connection to the database with `create_engine`.

>     engine = create_engine('sqlite:///states.db', echo=True)

Then we bind our new session to the engine to establish a connection to our database.

>     Session = sessionmaker(bind=engine)
>     session = Session()


### Instructions

#### Create the table

Create the `states` table in the `schema.py` file.  We have provided you with all of the code for configuring the database.  Every state has an `id` (set as the primary key), a `name`, a `capital_city`, a `population`, and a `landlocked` column that holds a boolean for whether the state borders a body of water.

Once this task is completed, move onto the `states_controller.py` file.  There is no need to run `python schema.py` in the terminal to create the database and schema since the tests automatically execute this file.

#### CREATE

We will write all of our CRUD actions in the `states_controller.py` file.  We provided the starter code for starting a session in the first method to get you going.

* New York
    - name: 'New York'
    - capital: 'Albany'
    - population: 20000000
    - landlocked: no
    
* Wyoming
    - name: 'Wyoming'
    - capital: 'Cheyenne'
    - population: 579315
    - landlocked: yes

In [1]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
class State(Base):
    __tablename__ = 'states'
    id = Column(Integer, primary_key = True)
    name = Column(Text)
    capital_city = Column(Text)
    population = Column(Integer)
    landlocked = Column(Boolean)
engine = create_engine('sqlite:///states.db', echo=True)

In [2]:
Base.metadata.create_all(engine)

2018-07-11 16:34:16,623 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-11 16:34:16,627 INFO sqlalchemy.engine.base.Engine ()
2018-07-11 16:34:16,631 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-11 16:34:16,633 INFO sqlalchemy.engine.base.Engine ()
2018-07-11 16:34:16,638 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("states")
2018-07-11 16:34:16,639 INFO sqlalchemy.engine.base.Engine ()


In [3]:
from sqlalchemy.orm import sessionmaker

In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
new_york = State(name = 'New York', capital_city = 'Albany', population = 20000000, landlocked = False)
session.add(new_york)
session.commit()
wyoming = State(name = 'Wyoming', capital_city = 'Cheyenne', population = 579315, landlocked = True)
session.add(wyoming)
session.commit()

2018-07-11 16:34:16,679 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-11 16:34:16,683 INFO sqlalchemy.engine.base.Engine INSERT INTO states (name, capital_city, population, landlocked) VALUES (?, ?, ?, ?)
2018-07-11 16:34:16,687 INFO sqlalchemy.engine.base.Engine ('New York', 'Albany', 20000000, 0)
2018-07-11 16:34:16,692 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-11 16:34:16,702 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-11 16:34:16,704 INFO sqlalchemy.engine.base.Engine INSERT INTO states (name, capital_city, population, landlocked) VALUES (?, ?, ?, ?)
2018-07-11 16:34:16,705 INFO sqlalchemy.engine.base.Engine ('Wyoming', 'Cheyenne', 579315, 1)
2018-07-11 16:34:16,707 INFO sqlalchemy.engine.base.Engine COMMIT


#### READ

* Query all states in the database

For now, we will just query all the data in our table.  We will explore more advanced queries in a later lesson.  Use `session.query()` and `.all()` to pull everything off of the table and set it to a variable.  Return this variable to get the test to pass.

In [9]:
Session = sessionmaker(bind=engine)
session = Session()
cali = State(name="California", capital_city="Sacramento", population=40000000, landlocked=False)
session.add(cali)
session.commit()

2018-07-11 16:35:17,157 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-11 16:35:17,159 INFO sqlalchemy.engine.base.Engine INSERT INTO states (name, capital_city, population, landlocked) VALUES (?, ?, ?, ?)
2018-07-11 16:35:17,160 INFO sqlalchemy.engine.base.Engine ('California', 'Sacramento', 40000000, 0)
2018-07-11 16:35:17,162 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
def create_cali():
    Session = sessionmaker(bind=engine)
    session = Session()
    cali = State(name="California", capital_city="Sacramento", population=40000000, landlocked=False)
    session.add(cali)
    session.commit()

In [26]:
cali = session.query(State).filter_by(name = 'California').first()
cali.population = 50000000
session.commit()

2018-07-11 16:48:19,958 INFO sqlalchemy.engine.base.Engine SELECT states.id AS states_id, states.name AS states_name, states.capital_city AS states_capital_city, states.population AS states_population, states.landlocked AS states_landlocked 
FROM states 
WHERE states.name = ?
 LIMIT ? OFFSET ?
2018-07-11 16:48:19,960 INFO sqlalchemy.engine.base.Engine ('California', 1, 0)
2018-07-11 16:48:19,962 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
Base.metadata.create_all(engine)

2018-07-11 16:34:16,777 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("states")
2018-07-11 16:34:16,781 INFO sqlalchemy.engine.base.Engine ()


#### UPDATE

First, uncomment the `create_cali()` function.  

Then in the `update_cali()` function:

* Find California by using `session.query()` in conjunction with `filter_by`.  You can read about the latter method [here](http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.filter_by).

* Change California's population to 50000000 then commit the update.

#### DELETE

First, uncomment the `create_connecticut()` function.  

Then in  the `delete_connecticut()`:

* Find the Connecticut object
* Delete it from the database 

### Session Rollbacks

Remember if we mistakenly change data during a session, we can rollback the error as long as we haven't already committed those changes to the database.  In the final method, uncomment the code for adding West Dakota to the table.  Write a rollback so that this "state" doesn't get added to the database when we commit the session.

## Summary

In this lab, we practiced setting up a connection to our database and creating a session. We then practiced defining CRUD functions that would create, read, update, and delete information from our database. 